## First, import the libraries we will use

In [1]:
%matplotlib inline

import numpy as np
from datetime import datetime, timedelta
from pyproj import Proj
import xarray
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from scipy import spatial

## Open the GOES-17 NetCDF File
Using xarray, I assign the opened file to the variable C for the CONUS domain.

In [2]:
#GOES 16 all in one file
#FILE = 'OR_ABI-L2-MCMIPC-M3_G16_s20190422102136_e20190422104509_c20190422105021.nc'

#GOES 17 on Amazon, each channel in a different file
FILE_5 ='OR_ABI-L2-CMIPC-M3C05_G17_s20190422102163_e20190422104536_c20190422105008.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190422102163_e20190422104536_c20190422104597.nc'
FILE_3 = 'OR_ABI-L2-CMIPM1-M3C03_G17_s20190461730268_e20190461730326_c20190461730375.nc'
File_5 = 'OR_ABI-L2-CMIPM1-M3C05_G17_s20190461730268_e20190461730326_c20190461730375.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190461732189_e20190461734562_c20190461735029.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190461732189_e20190461734562_c20190461735030.nc'
FILE_3 = 'OR_ABI-L2-CMIPM1-M3C03_G17_s20190461700268_e20190461700326_c20190461700370.nc'
FILE_5 = 'OR_ABI-L2-CMIPM1-M3C05_G17_s20190461700268_e20190461700326_c20190461700373.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190741802189_e20190741804562_c20190741805023.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190741802189_e20190741804562_c20190741805035.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190742102189_e20190742104562_c20190742105016.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190742102189_e20190742104562_c20190742105023.nc'
FILE_3 = 'OR_ABI-L2-CMIPC-M3C03_G17_s20190752002189_e20190752004562_c20190752005027.nc'
FILE_5 = 'OR_ABI-L2-CMIPC-M3C05_G17_s20190752002189_e20190752004562_c20190752005028.nc'
FILE_2 = 'OR_ABI-L2-CMIPC-M3C02_G17_s20190752002189_e20190752004562_c20190752005057.nc'

FILE_3_16 = 'OR_ABI-L2-CMIPC-M3C03_G16_s20190752002189_e20190752004562_c20190752005035.nc'
FILE_5_16 = 'OR_ABI-L2-CMIPC-M3C05_G16_s20190752002189_e20190752004562_c20190752005036.nc'


# goes 16
C_5_16 = xarray.open_dataset(FILE_5_16)
C_3_16 = xarray.open_dataset(FILE_3_16)

In [3]:
#goes 16 info
C_5_16['goes_imager_projection']

<xarray.DataArray 'goes_imager_projection' ()>
array(-2147483647)
Coordinates:
    t        datetime64[ns] ...
    y_image  float32 ...
    x_image  float32 ...
Attributes:
    long_name:                       GOES-R ABI fixed grid projection
    grid_mapping_name:               geostationary
    perspective_point_height:        35786023.0
    semi_major_axis:                 6378137.0
    semi_minor_axis:                 6356752.31414
    inverse_flattening:              298.2572221
    latitude_of_projection_origin:   0.0
    longitude_of_projection_origin:  -75.0
    sweep_angle_axis:                x

In [4]:
# Satellite height goes 16
sat_h_16 = C_5_16['goes_imager_projection'].perspective_point_height

# Satellite longitude
#sat_lon = C_5['goes_imager_projection'].longitude_of_projection_origin
#uncertainty whether at -75.2; -75 seems correct now
sat_lon_16 = -75

# Satellite sweep
sat_sweep_16 = C_5_16['goes_imager_projection'].sweep_angle_axis

# The projection x and y coordinates equals the scanning angle (in radians) multiplied by the satellite height
# See details here: https://proj4.org/operations/projections/geos.html?highlight=geostationary
x_16 = C_5_16['x'][:] * sat_h_16
y_16 = C_5_16['y'][:] * sat_h_16

In [5]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

projection_16 = ccrs.Geostationary(central_longitude=sat_lon_16, satellite_height=sat_h_16,
                               sweep_axis=sat_sweep_16)
img_extent_16 = (x_16.min(), x_16.max(), y_16.min(), y_16.max())




#### The magic function: `pyproj.Proj`
This function creates a map projection object of the geostationary projection.

In [6]:


# Create a pyproj geostationary map object
p_16 = Proj(proj='geos', h=sat_h_16, lon_0=sat_lon_16, sweep=sat_sweep_16)
# Perform cartographic transformation for goes 16. That is, convert image projection coordinates (x and y)
# to latitude and longitude values.
XX_16, YY_16 = np.meshgrid(x_16, y_16)
lons_16, lats_16 = p_16(XX_16, YY_16, inverse=True)
print('GOES 16 dimensions: %s %s' % np.shape(lons_16))

iy=len(lons_16[:,0])
ix=len(lons_16[0])
print(ix,iy)


GOES 16 dimensions: 3000 5000
5000 3000


In [7]:

mini=5000
minj=3000
maxi=0
maxj=0
clat = 40.5
clon = -113.
for i in range(0,iy):
    for j in range(0,ix):
        if(clon-1<lons_16[i,j]<clon+1):
            if(clat-1<lats_16[i,j]<clat+1):
                    if(i<mini):
                        mini=i
                    if(i>maxi):
                        maxi=i
                    if(j<minj):
                        minj=j
                    if(j>maxj):
                        maxj=j
print(mini,maxi,minj,maxj)
print(lons_16[mini,minj],lats_16[mini,minj],lons_16[maxi,maxj],lats_16[maxi,maxj])
mini=mini-50
if(mini<0):
    mini=0
minj=minj-50
if(minj<0):
    minj=0
maxi=maxi+50
if(maxi>ix):
    maxi=ix
maxj=maxj+50
if(maxj>iy):
    maxj=iy
ir=maxi-mini+1
jr=maxj-minj+1
lons_16t=np.zeros((ir,jr))
lats_16t=np.zeros((ir,jr))
ind_16t=np.zeros((ir*jr+1),dtype=int)
k=0
for i in range(0,ir):
    for j in range(0,jr):
        lons_16t[i,j]=lons_16[i+mini,j+minj]
        lats_16t[i,j]=lats_16[i+mini,j+minj]
        k=k+1
        ind_16t[k]=int(i*jr+j)
print(ind_16t)

680 833 753 958
-115.93193443114473 41.782815658212215 -110.3737292877345 39.26185105305145
[    0     0     1 ... 77721 77722 77723]


In [8]:
clat = 40.5
clon = -113.
lons_i = np.arange(-114.25, -111.75, 0.01)
lats_i = np.arange(39.25, 41.75, 0.01)
xx_i, yy_i = np.meshgrid(lons_i,lats_i)
dims_i = np.shape(xx_i)
iy = len(lats_i)
ix = len(lons_i)

In [9]:
lat_lon_16 = np.dstack([lats_16t.ravel(),lons_16t.ravel()])[0]
print(np.shape(lat_lon_16))
print(lat_lon_16)
lat_lon_i = np.dstack([yy_i.ravel(),xx_i.ravel()])[0]
print(np.shape(lat_lon_i))
print(lat_lon_i)
distance,index_16=spatial.KDTree(lat_lon_16).query(lat_lon_i)
print(index_16)

(77724, 2)
[[  42.64481791 -117.72180338]
 [  42.64306308 -117.70033343]
 [  42.64131    -117.67887664]
 ...
 [  38.51070577 -109.11385176]
 [  38.50968077 -109.09752511]
 [  38.50865653 -109.08120385]]
(62500, 2)
[[  39.25 -114.25]
 [  39.25 -114.24]
 [  39.25 -114.23]
 ...
 [  41.74 -111.78]
 [  41.74 -111.77]
 [  41.74 -111.76]]
[68567 68262 68263 ...  9766  9766  9767]


In [10]:
np.savez('proj_sf_16',index_16=index_16,ind_16t=ind_16t)